# Chapter 3  - Django Templates 

## Django Template Configuration

In [ ]:
import os

3.1. Default Django template configuration in settings.py


In [ ]:
TEMPLATES = [
    {
        'BACKEND': 'django.template.backends.django.DjangoTemplates',
        'DIRS': [],
        'APP_DIRS': True,
        'OPTIONS': {
            'context_processors': [
                'django.template.context_processors.debug',
                'django.template.context_processors.request',
                'django.contrib.auth.context_processors.auth',
                'django.contrib.messages.context_processors.messages',
            ],
        },
    },
]

3.2. Django apps with templates dirs with potential conflict and namespace qualification

Templates directly under templates folder can cause loading conflicts

```
+---+-<PROJECT_DIR_project_name_conflict>
    |
    +-__init__.py
    +-settings.py
    +-urls.py
    +-wsgi.py
    |
    +-about(app)-+
    |            +-__init__.py
    |            +-models.py
    |            +-tests.py
    |            +-views.py
    |            +-templates-+
    |                        |
    |                        +-index.html
    +-stores(app)-+
                  +-__init__.py
                  +-models.py
                  +-tests.py
                  +-views.py
                  +-templates-+
                              |
                              +-index.html`
```


Templates classified with additional namespace avoid loading conflicts

```
+---+-<PROJECT_DIR_project_name_conflict>
    |
    +-__init__.py
    +-settings.py
    +-urls.py
    +-wsgi.py
    |
    +-about(app)-+
    |            +-__init__.py
    |            +-models.py
    |            +-tests.py
    |            +-views.py
    |            +-templates-+
    |                        |
    |                        +-about-+
    |                                |
    |                                +-index.html
    +-stores(app)-+
                  +-__init__.py
                  +-models.py
                  +-tests.py
                  +-views.py
                  +-templates-+
                              |
                              +-stores-+
                                       |
                                       +-index.html`
```


3.3. DIRS definition with relative path in settings.py


In [ ]:
BASE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))
PROJECT_DIR = os.path.dirname(os.path.abspath(__file__))
TEMPLATES = [
    {
        #...
        'DIRS': [
            '%s/templates/' % (PROJECT_DIR),
            '%s/dev_templates/' % (PROJECT_DIR),
        ],
        #...
    },
]

3.4. Output warning message for invalid template variables with string_if_invalid


In [ ]:
TEMPLATES = [
    {
        #...
        'OPTIONS': {
            'string_if_invalid': "****WARNING: INVALID VARIABLE %s!!****",
            #...
        },
        #...
    },
]


3.5. Error generation for invalid template variables with string_if_invalid

In [ ]:
class InvalidTemplateVariable(str):
    def __mod__(self, other):
        from django.template.base import TemplateSyntaxError
        raise TemplateSyntaxError("Invalid variable : '%s'" % other)
    
TEMPLATES = [
    {
        #...
        'OPTIONS': {
            'string_if_invalid': InvalidTemplateVariable("%s"),
            #...
        },
    },
]


3.6. Option with debug equals False omits template details

In [ ]:
TEMPLATES = [
    {
        #...
        'OPTIONS': {
            'debug':False,
            #...
        },
    },
]

3.7. Option with auto-escape equals False omits auto-escaping on all Django templates

In [ ]:
TEMPLATES = [
    {
        #...
        'OPTIONS': {
            'autoescape':False,
            #...
        },
        #...
    },
]


3.8. Option with builtins to gain automatic access to tags/filters on all templates


In [ ]:
TEMPLATES = [
    {
        #...
        'OPTIONS': {
            #...
            'builtins': [
                'coffeehouse.builtins',
                'thirdpartyapp.customtags.really_useful_tags_and_filters',
            ],
        },
        #...
    },
]


3.9. Option with libraries to register tags/filters with alternative label/name and under any project directory

In [ ]:
TEMPLATES = [
    {
        #...
        'OPTIONS': {
            #...
            'libraries': {
                'coffeehouse_tags': 'coffeehouse.tags_filters.common',
            },
            #...
        },
        #...
    },
]

## Creating Reusable Templates

3.10. Django template with {% block %} tags

In [ ]:
<!DOCTYPE html>
<html lang="en">
    <head>
        <meta charset="utf-8">
        <title>{% block title%} Default title {% endblock title %}</title>
        <meta name="description" content="{% block metadescription%}{% endblock metadescription %}">
        <meta name="keywords" content="{% block metakeywords%}{% endblock metakeywords %}">


3.11. Django template with {% extends %} and {% block %} tag

In [ ]:
{% extends "base.html" %}
{% block title %}Coffeehouse home page{% endblock title %}

3.12. Django templates use of {{block.super}} with three reusable templates


In [ ]:
<!--base.html template-->

<p>{% block breadcrumb %}Home{% endblock breadcrumb %}</p>

<!--index.html template-->

{% extends "base.html" %}
{% block breadcrumb %}Main{% endblock breadcrumb %}

<!--detail.html template-->

{% extends "index.html" %}
{% block breadcrumb %} {{block.super}} : Detail {% endblock breadcrumb %}


# Custom Context processors

3.13. Custom Django context processor method

In [ ]:
def onsale(request):
    # Create fixed data structures to pass to template
    # data could equally come from database queries
    # web services or social APIs
    sale_items = {
        'Monday':'Mocha 2x1',
        'Tuesday':'Latte 2x1'
    }
    return {'SALE_ITEMS': sale_items}


3.14. Django template context processor definitions in context_processors in OPTIONS of TEMPLATES


In [ ]:
TEMPLATES = [
    {
        #...
        'OPTIONS': {
            'context_processors': [
                'coffeehouse.stores.processors.onsale',
                'django.template.context_processors.debug',
                'django.template.context_processors.request',
                'django.contrib.auth.context_processors.auth',
                'django.contrib.messages.context_processors.messages',
            ],
        },
        #...
    },
]


## Built-in Django Filters

3.15. Django linenumbers filter


In [ ]:
# Variable definition
'Downtown'
'Uptown'
'Midtown'

# Template definition with linenumbers filter

{{variable|linenumbers}}
# Output
'1.Downtown'
'2.Uptown'
'3.Midtown'

3.16. Django unordered_list filter

In [ ]:
# Variable definition
["Stores", ["San Diego", ["Downtown", "Uptown", "Midtown"]]]

# Template definition with linenumbers filter
{{variable|unordered_list}}

# Output
<li>Stores
    <ul>
        <li>San Diego
            <ul>
                <li>Downtown</li>
                <li>Uptown</li>
                <li>Midtown</li>
            </ul>
        </li>
    </ul>
</li>

3.17. Django wordwrap filter


In [ ]:
# Variable definition
Coffeehouse started as a small store

# Template definition with wordwrap filter for every 12 characters
{{variable|wordwrap:12}}

# Output
Coffeehouse
started as a
small store

## Built-in Django Tags

3.19. Django {% firstof %} tag and equivalent {% if %}{% elif %}{% else %} tags

In [ ]:
# Firstof example
{% firstof var1 var2 var3 %}

# Equivalent of firstof example
{% if var1 %}
    {{var1|safe}}
{% elif var2 %}
    {{var2|safe}}
{% elif var3 %}
    {{var3|safe}}
{% endif %}

# Firstof example with a default value in case of no match (i.e, all variables are empty)
{% firstof var1 var2 var3 "All vars are empty" %}

# Assign the firstof result to another variable
{% firstof var1 var2 var3 as resultof %}


3.20. Django {% for %} tag and {% for %} with {% empty %}


In [ ]:
<ul> 
    {% for drink in drinks %} 
        <li>{{ drink.name }}</li> 
        {% empty %} 
        <li>No drinks, sorry</li>
    {% endfor %}
 </ul>

<ul>
    {% for storeid, store in stores %}
        <li>
            <a href="/stores{{storeid}}/">{{store.name}}</a> 
        </li>
    {% endfor %}
</ul>


3.21. Django {% cycle %} with explicit control of progression


In [ ]:
<li class="{% cycle 'disc' 'circle' 'square' as bullettype %}">...</li>
<li class="{{bullettype}}">...</li>
<li class="{{bullettype}}">...</li>
<li class="{% cycle bullettype %}">...</li>
<li class="{{bullettype}}">...</li>
<li class="{% cycle bullettype %}">...</li>

# Outputs
<li class="disc">...</li>
<li class="disc">...</li>
<li class="disc">...</li>
<li class="circle">...</li>
<li class="circle">...</li>
<li class="square">...</li>

3.22. Django {% for %} tag and {% regroup %}

In [ ]:
# Dictionary definition
stores = [
    {'name': 'Downtown', 'street': '385 Main Street', 'city': 'San Diego'},
    {'name': 'Uptown', 'street': '231 Highland Avenue', 'city': 'San Diego'},
    {'name': 'Midtown', 'street': '85 Balboa Street', 'city': 'San Diego'},
    {'name': 'Downtown', 'street': '639 Spring Street', 'city': 'Los Angeles'},
    {'name': 'Midtown', 'street': '1407 Broadway Street', 'city': 'Los Angeles'},
    {'name': 'Downton', 'street': '50 1st Street', 'city': 'San Francisco'},
]
# Template definition with regroup and for tags
{% regroup stores by city as city_list %}

<ul>
    {% for city in city_list %}
        <li>{{ city.grouper }}
        <ul>
            {% for item in city.list %}
                <li>{{ item.name }}: {{ item.street }}</li>
            {% endfor %}
        </ul>
        </li>
    {% endfor %}
</ul>

# Output

San Diego
    Downtown : 385 Main Street
    Uptown : 231 Highland Avenue
    Midtown : 85 Balboa Street
Los Angeles
    Downtown: 639 Spring Street
    Midtown: 1407 Broadway Street
San Francisco
    Downtown: 50 1st Street


## Django Custom Filters

In [ ]:
from django import template
from django.utils.html import escape
from django.utils.safestring import mark_safe


3.23. Django custom filter with no arguments

In [ ]:
register = template.Library()

@register.filter()
def boldcoffee(value):
    '''Returns input wrapped in HTML tags'''
    return '<b>{}</b>'.format(value)

3.24. Django custom filter with arguments

In [ ]:
@register.filter()
def coffee(value, arg="muted"):
    '''Returns input wrapped in HTML tags with a CSS class'''
    '''Defaults to CSS class 'muted' from Bootstrap'''
    return '<span class="{}">{}</span>'.format(arg, value)


3.25. Django custom filter that detects autoescape setting

In [ ]:
register = template.Library()

@register.filter(needs_autoescape=True)
def smartcoffee(value, autoescape=True):
    '''Returns input wrapped in HTML tags'''
    '''and also detects surrounding autoescape on filter (if any) and escapes '''
    if autoescape:
        value = escape(value)
    result = '<b>{}</b>'.format(value)
    return mark_safe(result)


3.26. Django custom filter directory structure


```
+-<PROJECT_DIR_project_name>
|
+-__init__.py
+-settings.py
+-urls.py
+-wsgi.py
|
+----common----+
|              |
|              +--coffeehouse_filters.py
|
+---<app_one>--+
|              |
|              +-__init__.py
|              +-models.py
|              +-tests.py
|              +-views.py
|              +-----------<templatetags>---+
|                                           |
|                                           +-__init__.py
|                                           +-store_format_tf.py
+---<app_two>--+
|              |
               +-__init__.py
               +-models.py    
               +-tests.py
               +-views.py
               +-----------<templatetags>---+
                                            |
                                            +-__init__.py
                                            +-tax_operations.py
```

3.27. Configure Django template to load custom filters

In [ ]:
{% load store_format_tf %}
{% load store_format_tf tax_operations %}
{% load undercoffee from store_format_tf %}